In [48]:
from google.colab import drive
import sys
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir rutas base y repo
BASE_PATH = '/content/drive/MyDrive/Tesis'
REPO_NAME = 'Tesis_NN'
REPO_PATH = os.path.join(BASE_PATH, REPO_NAME)

# Agregar repo al path de Python para imports
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)

# Cambiar directorio actual al repo (para comandos git)
os.chdir(REPO_PATH)

print(f"Drive montado. Carpeta repo seteada en: {REPO_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive montado. Carpeta repo seteada en: /content/drive/MyDrive/Tesis/Tesis_NN


In [49]:
from google.colab import files
%cd /content/drive/MyDrive/Tesis/Tesis_NN
!pip install -r requirements.txt

/content
  Using cached asttokens-3.0.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached debugpy-1.8.14-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Using cached decorator-5.2.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached executing-2.2.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached fonttools-4.58.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (106 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached ipython-9.3.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached ipython_pygments_lexers-1.1.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl.metadata (8

In [50]:
!pip install webdataset

In [51]:
# ───────────────────────────────────────────────────────────────────────────────
# 📦 Standard Library Imports
# ───────────────────────────────────────────────────────────────────────────────
import os
import sys
import time
import csv
import json
from pathlib import Path

# ───────────────────────────────────────────────────────────────────────────────
# 📚 Scientific & Data Libraries
# ───────────────────────────────────────────────────────────────────────────────
import numpy as np
import matplotlib.pyplot as plt

# ───────────────────────────────────────────────────────────────────────────────
# 🌍 Third-Party Library Imports (PyTorch)
# ───────────────────────────────────────────────────────────────────────────────
import torch
import torch.optim as optim

# ───────────────────────────────────────────────────────────────────────────────
# 🧩 Custom Project Modules
# ───────────────────────────────────────────────────────────────────────────────
sys.path.append('C:/Users/nnobi/Desktop/FIUBA/Tesis/Project')

from project_package.utils import train_common_routines as tcr
from project_package.conv_net.ConvNet_model import SRCNN
from project_package.dataset_manager.webdataset_dataset import PtWebDataset



In [52]:
# ───────────────────────────────────────────────────────────────────────────────
# 🔧 Configuration
# ───────────────────────────────────────────────────────────────────────────────
model_selection = 'large'
epochs = 200
lr = 1e-5
batch_size = 32

# ───────────────────────────────────────────────────────────────────────────────
# 📁 Paths Setup
# ───────────────────────────────────────────────────────────────────────────────
dataset = 'dataset2' # Select Dataset
dataset_folder = os.path.join(REPO_PATH, 'datasets', dataset)
# tar_path = "file:" + dataset_folder.replace("\\", "/")
tar_path = dataset_folder.replace("\\", "/")
metadata_path = os.path.join(dataset_folder, 'metadata.json')
# metadata_path = '/content/drive/MyDrive/Tesis/Tesis_NN/datasets/messi/metadata.json'

with open(metadata_path, "r") as f:
    metadata = json.load(f)

train_samples = metadata["splits"]["train"]["num_samples"]
val_samples = metadata["splits"]["val"]["num_samples"]
test_samples = metadata["splits"]["test"]["num_samples"]

# Results folder and files
results_folder = os.path.join(REPO_PATH, 'results', 'Conv_Net')
os.makedirs(results_folder, exist_ok=True)

loss_png_file = os.path.join(results_folder, f"loss_lr={lr}_batch_size={batch_size}_model={model_selection}.png")
psnr_png_file = os.path.join(results_folder, f"psnr_lr={lr}_batch_size={batch_size}_model={model_selection}.png")
final_model_pth_file = os.path.join(results_folder, f"model_lr={lr}_batch_size={batch_size}_model={model_selection}.pth")
file_training_losses = os.path.join(results_folder, f"training_losses_lr={lr}_batch_size={batch_size}_model={model_selection}.csv")

In [53]:
# ───────────────────────────────────────────────────────────────────────────────
# 🚀 Training Pipeline
# ───────────────────────────────────────────────────────────────────────────────

# 🧠 Model Initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model = SRCNN(model_selection).to(device)
print("The model:")
print(model)

model.count_parameters()
print(f"Total Parameters: {model.total_params:,}")
print(f"Trainable Parameters: {model.trainable_params:,}")

model = tcr.multi_GPU_training(model)
optimizer = optim.Adam(model.parameters(), lr=lr)

# 📊 Dataset and DataLoaders
dataset_train = PtWebDataset(tar_path + "/train-*.tar", length=train_samples, batch_size=32, shuffle_buffer=10 * batch_size)
dataset_val = PtWebDataset(tar_path + '/val-*.tar', length=val_samples, batch_size=16, shuffle_buffer=2 * batch_size)
dataset_test = PtWebDataset(tar_path + '/test.tar', length=test_samples, batch_size=2, shuffle_buffer=2 * batch_size)

dataloader_train = dataset_train.get_dataloader(num_workers=2)
dataloader_val = dataset_val.get_dataloader(num_workers=0)
dataloader_test = dataset_test.get_dataloader(num_workers=0)

Device: cuda
The model:
SRCNN(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), padding_mode=replicate)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
  (relu3): ReLU(inplace=True)
  (conv4): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
)
Total Parameters: 427,139
Trainable Parameters: 427,139
Model training to be done in only one GPU!


/usr/local/lib/python3.11/dist-packages/webdataset/compat.py:389: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/webdataset/pipeline.py:176: UserWarning: .with_length() only sets the value of __len__ for compatibility with some training environments. It does not change the number of samples in an epoch.
  warnings.warn(


In [54]:
import glob
print(glob.glob('/content/drive/MyDrive/Tesis/Tesis_NN/datasets/dataset2/train-*.tar'))


['/content/drive/MyDrive/Tesis/Tesis_NN/datasets/dataset2/train-00000.tar', '/content/drive/MyDrive/Tesis/Tesis_NN/datasets/dataset2/train-00001.tar', '/content/drive/MyDrive/Tesis/Tesis_NN/datasets/dataset2/train-00003.tar', '/content/drive/MyDrive/Tesis/Tesis_NN/datasets/dataset2/train-00002.tar']


In [58]:
hola = PtWebDataset(tar_path + "/train-*.tar", length=train_samples, batch_size=32, shuffle_buffer=10 * batch_size)
hola.tar_path

/usr/local/lib/python3.11/dist-packages/webdataset/compat.py:389: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/webdataset/pipeline.py:176: UserWarning: .with_length() only sets the value of __len__ for compatibility with some training environments. It does not change the number of samples in an epoch.
  warnings.warn(


'/content/drive/MyDrive/Tesis/Tesis_NN/datasets/dataset2/train-*.tar'

In [ ]:
# 🏋️ Training Loop
train_loss, val_loss = [], []
train_psnr, val_psnr = [], []

start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1} of {epochs}")

    train_epoch_loss, train_epoch_psnr = tcr.train(model, dataloader_train, optimizer, tcr.compute_loss_MSE, device, train_samples)
    val_epoch_loss, val_epoch_psnr = tcr.validate(model, dataloader_val, epoch, tcr.compute_loss_MSE, device, val_samples)

    print(f"Train PSNR: {train_epoch_psnr:.3f}")
    print(f"Val PSNR: {val_epoch_psnr:.3f}")

    train_loss.append(train_epoch_loss)
    train_psnr.append(train_epoch_psnr)
    val_loss.append(val_epoch_loss)
    val_psnr.append(val_epoch_psnr)

    if epoch % 5 == 0:
        checkpoint_path = os.path.join(results_folder, f"checkpoint_epoch_{epoch}_lr={lr}_batch_size={batch_size}_model={model_selection}.pth")
        tcr.save_checkpoint(epoch, model, optimizer, train_loss, filename=checkpoint_path)

    with open(file_training_losses, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([train_epoch_loss, train_epoch_psnr, val_epoch_loss, val_epoch_psnr])

end = time.time()
print(f"\n✅ Finished training in: {(end - start) / 60:.2f} minutes")

# 📈 Loss Plot
plt.figure(figsize=(10, 7))
plt.plot(10 * np.log10(train_loss), color='orange', label='Train Loss')
plt.plot(10 * np.log10(val_loss), color='red', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (dB)')
plt.legend()
if os.path.exists(loss_png_file):
    os.remove(loss_png_file)
plt.savefig(loss_png_file)
plt.show()

# 📈 PSNR Plot
plt.figure(figsize=(10, 7))
plt.plot(train_psnr, color='green', label='Train PSNR (dB)')
plt.plot(val_psnr, color='blue', label='Validation PSNR (dB)')
plt.xlabel('Epochs')
plt.ylabel('PSNR (dB)')
plt.legend()
if os.path.exists(psnr_png_file):
    os.remove(psnr_png_file)
plt.savefig(psnr_png_file)
plt.show()

# 💾 Save Final Model
print('\n💾 Saving model...')
if os.path.exists(final_model_pth_file):
    os.remove(final_model_pth_file)
model = model.module if hasattr(model, "module") else model
torch.save(model.state_dict(), final_model_pth_file)



Epoch 1 of 200


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 42, in fetch
    data = next(self.dataset_iter)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/webdataset/pipeline.py", line 105, in iterator
    for sample in self.iterator1():
  File "/usr/local/lib/python3.11/dist-packages/webdataset/filters.py", line 799, in _batched
    for sample in data:
  File "/usr/local/lib/python3.11/dist-packages/webdataset/filters.py", line 358, in _shuffle
    for sample in data:
  File "/usr/local/lib/python3.11/dist-packages/webdataset/filters.py", line 701, in _map_tuple
    for sample in data:
  File "/usr